In [ ]:
###### Model Deployment in Sagemaker #########
##############################################
import boto3
from sagemaker.model import Model
from sagemaker import get_execution_role, Session 

role = get_execution_role()
session = Session()

ecr_image = '491085388405.dkr.ecr.us-east-1.amazonaws.com/fraud-ecr-1908:v1'

model = Model(
    image_uri=ecr_image,
    role=role,
    sagemaker_session=session 
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name='fraud-ml-endpoint'
)

In [2]:
######## Inference using one row of Sample Data #########
#########################################################
import boto3
import json

# Create SageMaker runtime client
client = boto3.client('sagemaker-runtime', region_name='us-east-1')  # update region if needed

# Sample input — should match model's expected feature shape and order
payload = {
    "inputs": [[18.0, 1.0, 100987.0, 120870.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0]]
}

# Invoke the deployed endpoint
response = client.invoke_endpoint(
    EndpointName='fraud-ml-endpoint',  # Make sure this matches exactly
    Body=json.dumps(payload),
    ContentType='application/json'
)

# Read and decode the prediction
result = response['Body'].read().decode('utf-8')
print("Prediction:", result)


Prediction: {"predictions":[0]}



In [3]:
######## Inference using two rows of Sample Data #########
##########################################################
import boto3
import json

# Create SageMaker runtime client
client = boto3.client('sagemaker-runtime', region_name='us-east-1')  # update region if needed

# Sample input — should match model's expected feature shape and order
# payload = {
#     "inputs": [[18.0, 1.0, 100987.0, 120870.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0]]
# }

payload = {
    "inputs": [
        [18.0, 1.0, 100987.0, 120870.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0],
        [25.0, 0.0, 200000.0, 150000.0, 1.0, 3.0, 0.0, 1.0, 0.0, 1.0]
    ]
}

# Invoke the deployed endpoint
response = client.invoke_endpoint(
    EndpointName='fraud-ml-endpoint',  # Make sure this matches exactly
    Body=json.dumps(payload),
    ContentType='application/json'
)

# Read and decode the prediction
result = response['Body'].read().decode('utf-8')
print("Prediction:", result)


Prediction: {"predictions":[0,0]}



In [5]:
############ To create V2 of the End-point ##########
#####################################################

import boto3

# Initialize SageMaker client (region must match your ECR/SageMaker)
sm_client = boto3.client("sagemaker", region_name="us-east-1")

role = get_execution_role()


response = sm_client.create_model(
    ModelName="fraud-model-v2",
    PrimaryContainer={
        "Image": "491085388405.dkr.ecr.us-east-1.amazonaws.com/fraud-ecr-1908:v2"
    },
    ExecutionRoleArn=role
)
print("Model v2 Created:", response["ModelArn"]) 


Model v2 Created: arn:aws:sagemaker:us-east-1:491085388405:model/fraud-model-v2


In [6]:
###### To check what are the model names available in the Endpoint ########
###########################################################################
import boto3

sm_client = boto3.client("sagemaker", region_name="us-east-1")

response = sm_client.list_models(MaxResults=10)
for model in response["Models"]:
    print(model["ModelName"])


fraud-model-v2
fraud-ecr-1908-2025-08-19-08-21-21-915


In [8]:
########### A/B testing Configuration ##########
################################################

import boto3

sm_client = boto3.client("sagemaker")

endpoint_config_name = "fraud-endpoint-config-ab"

response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "Variant1",
            "ModelName": "fraud-model-v2",   # your v2 model
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m5.large",
            "InitialVariantWeight": 0.7,   # 70% traffic
        },
        {
            "VariantName": "Variant2",
            "ModelName": "fraud-ecr-1908-2025-08-19-08-21-21-915",   # your ECR model
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m5.large",
            "InitialVariantWeight": 0.3,   # 30% traffic
        },
    ],
)

print("Created Endpoint Config:", endpoint_config_name)


Created Endpoint Config: fraud-endpoint-config-ab


In [9]:
###### create or update the endpoint to use this new endpoint config (fraud-endpoint-config-ab)######
#####################################################################################################
import boto3

sm_client = boto3.client("sagemaker")

endpoint_name = "fraud-endpoint"

response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName="fraud-endpoint-config-ab"
)

print("✅ Endpoint created:", response['EndpointArn'])


✅ Endpoint created: arn:aws:sagemaker:us-east-1:491085388405:endpoint/fraud-endpoint


In [10]:
######## Inference using two rows of Sample Data #########
##########################################################
import boto3
import json

# Create SageMaker runtime client
client = boto3.client('sagemaker-runtime', region_name='us-east-1')  # update region if needed

# Sample input — should match model's expected feature shape and order
# payload = {
#     "inputs": [[18.0, 1.0, 100987.0, 120870.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0]]
# }

payload = {
    "inputs": [
        [18.0, 1.0, 100987.0, 120870.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0],
        [25.0, 0.0, 200000.0, 150000.0, 1.0, 3.0, 0.0, 1.0, 0.0, 1.0]
    ]
}

# Invoke the deployed endpoint
response = client.invoke_endpoint(
    EndpointName='fraud-ml-endpoint',  # Make sure this matches exactly
    Body=json.dumps(payload),
    ContentType='application/json'
)

# Read and decode the prediction
result = response['Body'].read().decode('utf-8')
print("Prediction:", result)


Prediction: {"predictions":[0,0]}



In [12]:
# file: model_monitoring_dashboard.py
%% writefile streamlit_dashboard.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# --------------------------
# Simulated Functions (replace with real ones later)
# --------------------------
def load_model_metrics():
    """Simulate loading model performance metrics (from logs/DB)."""
    data = {
        "timestamp": pd.date_range(datetime.now() - timedelta(days=30), periods=30),
        "accuracy": np.random.uniform(0.7, 0.95, 30),
        "f1_score": np.random.uniform(0.6, 0.9, 30),
    }
    return pd.DataFrame(data)

def load_data_drift():
    """Simulate loading data drift statistics."""
    features = ["feature1", "feature2", "feature3", "feature4"]
    drift_scores = np.random.uniform(0, 1, len(features))
    return pd.DataFrame({"feature": features, "drift_score": drift_scores})

# --------------------------
# Streamlit Dashboard
# --------------------------
st.set_page_config(page_title="MLOps Monitoring Dashboard", layout="wide")

st.title("📊 Model & Data Drift Monitoring Dashboard")

# Section 1: Model Performance
st.subheader("Model Performance Over Time")
metrics_df = load_model_metrics()

col1, col2 = st.columns(2)

with col1:
    st.line_chart(metrics_df.set_index("timestamp")[["accuracy"]], height=250)
    st.caption("Model Accuracy trend")

with col2:
    st.line_chart(metrics_df.set_index("timestamp")[["f1_score"]], height=250)
    st.caption("Model F1-score trend")

# Section 2: Data Drift
st.subheader("Data Drift by Feature")
drift_df = load_data_drift()

st.bar_chart(drift_df.set_index("feature"), height=300)

# Section 3: Alerts
st.subheader("🚨 Alerts")
drift_threshold = 0.7
alerts = drift_df[drift_df["drift_score"] > drift_threshold]

if not alerts.empty:
    st.error(f"High drift detected in features: {', '.join(alerts['feature'])}")
else:
    st.success("No significant data drift detected.")

# Section 4: Model Registry (placeholder)
st.subheader("Model Registry & Versions")
st.write("Here you can integrate with SageMaker Model Registry to fetch version info.")
st.table(
    pd.DataFrame({
        "Model Version": ["v1", "v2", "v3"],
        "Deployed At": [
            datetime.now() - timedelta(days=20),
            datetime.now() - timedelta(days=10),
            datetime.now() - timedelta(days=2),
        ],
        "Status": ["Archived", "Production", "Staging"],
    })
)

st.caption("This is a starter dashboard. Integrate it with SageMaker Model Monitor, CloudWatch logs, or your metadata DB.")


UsageError: Line magic function `%%` not found.


In [13]:
%%writefile dashboard.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import datetime

# Simulated function to fetch model metrics
def get_model_metrics():
    return {
        "accuracy": np.random.uniform(0.7, 0.99),
        "precision": np.random.uniform(0.6, 0.95),
        "recall": np.random.uniform(0.5, 0.9)
    }

# Simulated function to detect drift
def detect_data_drift():
    return np.random.choice([True, False], p=[0.2, 0.8])

st.title("🚀 MLOps Continuous Monitoring Dashboard")
st.markdown("Monitoring **Model Performance** and **Data Drift** in Real-Time")

# Model Metrics Section
st.header("📊 Model Performance")
metrics = get_model_metrics()
st.write("Latest Metrics:", metrics)

st.line_chart(pd.DataFrame({
    "accuracy": [metrics["accuracy"]],
    "precision": [metrics["precision"]],
    "recall": [metrics["recall"]]
}))

# Drift Detection Section
st.header("⚠️ Data Drift Detection")
drift_detected = detect_data_drift()

if drift_detected:
    st.error("Data Drift Detected! 🚨 Retraining may be required.")
else:
    st.success("No significant drift detected ✅")

# Logs Section
st.header("📝 Monitoring Logs")
log_data = pd.DataFrame({
    "timestamp": [datetime.datetime.now()],
    "accuracy": [metrics["accuracy"]],
    "drift_detected": [drift_detected]
})
st.table(log_data)


Writing dashboard.py


In [15]:
!pip install streamlit --quiet

In [16]:
!streamlit run dashboard.py --server.port 8501 --server.address 0.0.0.0




  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

^C
  Stopping...


In [ ]:
!streamlit run dashboard.py --server.port 8501 --server.address 0.0.0.0





  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

